In [ ]:
import pandas as pd
from  tqdm.notebook import tqdm
from shapely import wkt
import geopandas
import matplotlib.pyplot as plt
import os

# Eksploracja danych nypd-motor-vehicle-collisions.csv

## Wczytanie danch
Wczytanie danych o wypadkach i kolizjach w mieście Nowy Jork z pliku csv.
Plik posiada nagłówek z nazwami kolumn a jako separatora użyto znaku ','

In [ ]:
# Wczytuję dana z pliku csv
ny_collisions = pd.read_csv("data/nypd-motor-vehicle-collisions.csv")

In [ ]:
ny_collisions.head(3)

In [ ]:
# Z powodu komunikatu Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
# przyglądam się kolumnie (3) "ZIP CODE"
ny_collisions["ZIP CODE"]

Pandas potraktował wartości kolumny ZIP CODE jako liczby. Bezpośrednio w pliku CSV jest to głównie 5 cyfr, ponadto kolumna posiada wartości puste lub wypełnione białymi znakami.
Traktuję kolumnę **ZIP CODE** jako ciąg znaków (typ str) a kolumnę **ACCIDENT DATE** jako dane typu datetime i wczytuję dane ponownie.

In [ ]:
# kolumna ZIP CODE jako str, a ACCIDENT DATE jako datetime
ny_collisions = pd.read_csv("data/nypd-motor-vehicle-collisions.csv",dtype={"ZIP CODE":'str'}, parse_dates=["ACCIDENT DATE"])
ny_collisions.head(3)

## Podstawowe informacje o wczytanym DataFrame
Wczytany DataFrame ny_collisions posiada 1 612 178 wierszy w 29 kolumnach.

Dane o kolumnach DataFrame ny_collisions:
* **ACCIDENT DATE**: Data wypadku, podczas wczytywania kolumna przekształcona na typ danych 'datetime64[ns]'
* **ZIP CODE**: Kod pocztowy, typ danych ciąg znaków 'str'

In [ ]:
# Podstawowe informacje o DataFrame ny_colision
# 1612178 wierszy i 29 kolumn
ny_collisions.shape

In [ ]:
ny_collisions.describe

In [ ]:
ny_collisions.info

In [ ]:
# Nazwy kolumn DataFrame ny_colision
ny_collisions.columns

In [ ]:
# Typy danych kolumn
print("ACCIDENT DATE: ",ny_collisions["ACCIDENT DATE"].dtype)
print("ZIP CODE: ",ny_collisions["ZIP CODE"].dtype)
print("COLLISION_ID: ",ny_collisions["COLLISION_ID"].dtype)
print("LATITUDE: ",ny_collisions["LATITUDE"].dtype)
print("LONGITUDE: ",ny_collisions["LONGITUDE"].dtype)
print("LOCATION: ",ny_collisions["LOCATION"].dtype)

## Exploracja danych w poszczególnych kolumnach
### Kolumna COLLISION_ID
Sprawdzam czy kolumnę COLLISION_ID można użyć jako klucza głównego DataFrame, jednoznacznie wskazującego wiersz

In [ ]:
# typ danych
ny_collisions["COLLISION_ID"].dtype

In [ ]:
# czy są komórki nie uzupełnione
ny_collisions["COLLISION_ID"].isna().sum()

In [ ]:
# Czy wartości w kolumnie są unikalne ?
cnt_no_uniq = (ny_collisions["COLLISION_ID"].value_counts() > 1).sum()
cnt_no_uniq
# Niestety nie, jest jak poniżej pewna liczba zduplikowanych wartości w kolumnie COLLISION_ID

In [ ]:
# rozpiętość ilości nie unikalnych wartości w kolumnie COLLISION_ID
ny_collisions["COLLISION_ID"].value_counts().agg(["min","max"])
# Kolumna posiada maksymalnie dwie powielone wartości w COLLISION_ID.

Wyświetlam wiersze DataFrame z powielonymi wartościami w kolumnie COLLISION_ID

In [ ]:
coll_id_s = ny_collisions["COLLISION_ID"].value_counts() > 1
coll_id_s[coll_id_s].index
# dataframe ny_collisions z powielonymi wartościami w kolumnie COLLISION_ID

In [ ]:
# Wyświetlam jeden z wierzy aby przyjżeć się kolumnom 
ny_collisions.loc[ny_collisions["COLLISION_ID"] == 3126615]

Z 2 wierszy wyświetlonych powyżej wygląda że kolumny prawdopodobnie mają te same wartości w wszystkich dwóch wierszach i będzie można wyczyścić dane w DataFrame Korzystając z funkcji drop_duplicates(). Najpierw trochę zabawy z funkcją sprawdzającą czy kolumny w dataframe z zduplikowanymi wartościami COLLISION_ID, posiadją te same wartości.

In [ ]:
# "COLLISION_ID"
# Funkcja grupuje DataFrame df po kolumnie col, wyszukuje grupy które zawierają więcej niż 1 wiersz i
# sprawdza czy wartości w zgrupowanych kolumnach są różne.
# Funkcja zwraca słownik gdzie klucz to wartość grupowanej kolumny, a wrtość to lista kolumn, które się różnią
# W przypadku gdy funkcja zwróci pusty słownik, dataframe df albo nie posiada zduplikowanych wierszy albo wszystkie kolumny
# w zduplikowanych wierszach posiadają identyczne wartości
def no_unique_columns(df,col):
    no_uniq = {} # {zduplikowana_wartość:[nazwa_kolumy_1_z_róznymi_wartościami,nazwa_kolumy_2_z_róznymi_wartościami,...}
    for g in tqdm(df.groupby(col)):
        if len(g[1]) > 1:
            # Jeżeli są duplikaty badamy wartości w odpowiednich kolumnach
            for c in g[1].columns:
                if len(g[1][c].unique()) > 1:
                    # wartość zduplikowana (g[0]), posiada rózne wartości w dopowiednich kolumnach. Dodajemy informacje do słownika
                    if g[0] in no_uniq:
                        no_uniq[g[0]].append(c)
                    else:
                        no_uniq[g[0]] = [c]
    return no_uniq

In [ ]:
# test unikalności wartości. Chwilkę może potrwać ... (4 min.)
nuq = no_unique_columns(ny_collisions,"COLLISION_ID")
nuq

In [ ]:
# Wszystkie wiersze z zdupliownymi wartościami COLLISION_ID mają również zduplikowane wartości w wszystkich kolumnach.
# Czyszczę DataFrame
cnt_before = len(ny_collisions)
ny_collisions = ny_collisions.drop_duplicates("COLLISION_ID")
cnt_after = len(ny_collisions)
print(f"Ilość wierszy przed czyszczeniem: {cnt_before}\nIlość wierszy po wyczyszczeniu: {cnt_after}\nIlość wierszy usuniętych: {cnt_before-cnt_after}")
print(f"Wierszy z nieunikalnymi wartościami w kolumnie COLLISION_ID było {cnt_no_uniq}")

In [ ]:
# Liczba zduplikowanych wartości w kolumnie COLLISION_ID
(ny_collisions["COLLISION_ID"].value_counts() > 1).sum()
# Kolumny można użyć jako klucza głównego, jednoznacznie identyfikującego wiersze DataFrame

### Kolumna "ZIP CODE"

[Kod pocztowy USA](https://en.wikipedia.org/wiki/ZIP_Code) w podstawowej formie składa się z 5 cyfr, ale może posiadać więcej niż 5 znaków.
Ponadto kod pocztowy stanu NY rozpoczyna się od znaku '1'.

In [ ]:
# Wyczyszczenie kolumny z początkowych i końcowych białych znków
ny_collisions.loc[:,"ZIP CODE"] = ny_collisions["ZIP CODE"].str.strip()

Liczba wierszy w których kolumna "ZIP CODE" posiada wartość nieuzupełnioną wynosi:

In [ ]:
# liczba wierszy z kolumną ZIP CODE równym NaN lub pystym ciągiem znanków ''
# Jeżeli kolumna ZIP CODE w pliku csv jest w postaci ,, to ma wartość NaN, jeżeli ,\s+, (,jedna lub więcej znaków białych,) ma wartość ''
count_zip_code_nan = len(ny_collisions.loc[ny_collisions["ZIP CODE"].isna() | (ny_collisions["ZIP CODE"] == '')])
count_zip_code_nan

Zgodnie z informacją o kodach pocztowych w USA, kdoy NYC powinny rozpoczynać się od znaku cyfry jeden ('1').
Liczba kodów pocztowych w kolumnie "ZIP CODE" w formacie podstawowym (5 znaków) i rozpoczynająca się od znaku cyfry '1' wynosi:

In [ ]:
# Liczba wierszy w której kolumna ZIP CODE składa się z 5 cyfr, przyczym pierwsza rozpoczyna się od znaku '1'
count_zip_code_basic = ny_collisions["ZIP CODE"].str.count("1\d{4}").sum()
count_zip_code_basic

Suma ilości wierszy w której kolumna ZIP CODE ma wartość NaN i poprawny ZIP CODE stanu NY rozpoczynający się od
znaku '1', składjący się z 5 cyfr, powinna być równa ilości wierszy DataFrame ny_collision

In [ ]:
len(ny_collisions) - count_zip_code_nan - count_zip_code_basic

Operacje poniżej normalizują kolumnę ZIP CODE, tylko i wyłącznie do typu str, dla późniejszych analiz. Pozbywam się wartości NaN przez zamianę jej na pusty ciąg znaków ''.

In [ ]:
ny_collisions.loc[ny_collisions["ZIP CODE"].isna(),"ZIP CODE"] = ''

In [ ]:
len(ny_collisions["COLLISION_ID"].loc[ny_collisions["ZIP CODE"]==''])

### Kolumny "LATITUDE", "LONGITUDE", "LOCATION"

Sprawdzam jak wypełnieone są zależne od siebie kolumny LATITUDE, LONGITUDE i LOCATION.

Przypadki:
* wszystkie trzy komórki w wierszu dla poszczególnych kolumn posiadają dane
* wszystkie trzy komórki w wierszu dla poszczególnych kolumny są nieuzupełnione
* część komórek w wierszu dla poszczególnych kolumn jest wypełniona a część nie

In [ ]:
# Wyświetlenie nieuzupełnionych kolumn "LATITUDE","LONGITUDE","LOCATION"
# False: komórka w kolumnie uzupełniona, True: komórka w kolumnie nie uzupełniona
ny_collisions[["LATITUDE","LONGITUDE","LOCATION"]].isna()

Wartość False komórki w odpowiadającej kolumnie oznacza że jest uzupełniona, True nie uzupełniona

Sumowanie po osi X wartości boolen (False=0, True=1) w celu określenia statusu uzupełnienia kolumn.

Znczenie sumy wartości logicznych trzech kolumn "LATITUDE","LONGITUDE","LOCATION" w poszczególnych wierszach:

0 - wszystkie kolumny zostały wypełnione\
1 - jedna z trzech komórek wiersza nie została uzupełniona\
2 - dwie z trzech komórek wiersza nie zostały uzupełnione\
3 - wszystkie trzy komórki w wierszu nie zostały uzupełnione

In [ ]:
ny_collisions[["LATITUDE","LONGITUDE","LOCATION"]].isna().sum(axis=1)

Sprawdzenie z jakiego typu "brakami" w danych o lokalizacji kolizji/wypadku mamy doczynienia

In [ ]:
(ny_collisions[["LATITUDE","LONGITUDE","LOCATION"]].isna().sum(axis=1)).unique()

In [ ]:
# liczba wierszy bez współrzędnych geograficznych
((ny_collisions[["LATITUDE","LONGITUDE","LOCATION"]].isna().sum(axis=1)) == 3).sum()

W tym przypadku posiadamy tylko wartości 0 i 3 co oznacza że mamy wiersze w kolumnach "LATITUDE","LONGITUDE","LOCATION" albo wszystkie uzupełnione albo wszystkie nie uzupełnione. Brak przypadków mieszanych (jedna lub dwie kolumny są uzupełnione a reszta nie), co oszczędzi głębszej analizy, które komórki można uzupełnić na podstawie zawartości inych komórek (np. LATITUDE na podstawie uzupełnionej komórki LOCATION).

Komórka LOCATION zapisana jest jako typ danych str i wygląda jak rekord danych python dictionary:\
`{'type': 'Point', 'coordinates': [-73.790184, 40.676052]}`\
gdzie klucz 'type' określa prawdopodobnie kształt (w tym przypadku punkt) i 'coordinates' współrzędne geograficzne, 
co daje możliwość weryfikacji spójności danych komórek w kolumnach LATITUDE i LONGITUDE na podstawie informacji zawartych w komórkach kolumny LOCATION.

W celu wykonania takiej weryfikacji należy z komórki kolumny LOCATION "wyciągnąć" informacje o współrzędnych geograficznych.
Aby to osiągnąć można potraktować zawartość wierszy kolumny LOCATION jako dane typu json, zamieniając znak ' na znak ", lub bezpośrednio zamienić funkcją eval() ciąg znaków na słownik.

In [ ]:
# Funkcja sprawdza czy współrzędne geograficzne w kolumnie LATITUDE i LONGITUDE są zgodne z
# współrzędnymi geograficznymi w strukturze zawartej w kolumnie LOCATION
# Przeznaczenie: dla DataFrame.apply()
# wejście: wiersz danych dataframe z kolumnami LATITUDE,LONGITUDE,LOCATION
# wyjście: zwraca True jeżeli współrzędne geograficzne w strukturze z kolumny LOCATION są niezgodene z danymi w kolumnach LATITUDE i LONGITUDE
def check_coordinate(row):
    try:
        data = eval(row["LOCATION"])
        # Sprawdzam czy zmienna data jest typu słownik i czy słownik zawiera klucze type i coordinates
        if isinstance(data,dict) and (data.keys() >= {"type", "coordinates"}):
            if data["type"].strip().upper() == 'POINT':
                # Sprawdzanie czy klucz 'coordinates' zawiera listę z przynjmniej dwoma elementami nie ma sensu, zdziała wyjątek
                loc_latitude = data["coordinates"][1]
                loc_longitude = data["coordinates"][0]
                return not ((loc_latitude-row["LATITUDE"]==0) and (loc_longitude-row["LONGITUDE"]==0))
    except:
        return True
    return True

In [ ]:
# tworzę kopię z dataframe ny_collision, która jest wycinkiem z oryginalnego dataframe składającego się z 
# kolumn "LATITUDE","LONGITUDE","LOCATION" bez LOCATION = NaN.
test = ny_collisions[["LATITUDE","LONGITUDE","LOCATION"]].loc[~ny_collisions["LOCATION"].isna()]

In [ ]:
# Testowanie zgodności wartości LONGITUDE, LATITUDE i LOCATION.coordinates.
test["check"] = test.apply(check_coordinate,axis=1)
test

In [ ]:
# Jeżeli są nizgodności to wiersz w kolumnie check ma wartość True
test.loc[test["check"]]

W tym przypadku brak niezgodności pomiędzy LATITUDE,LONGITUDE a kluczem coordinates w LOCATION. 
Z powyższej analizy wynika że kolumna LOCATION w dataframe ny_collision jest nadmiarowa.

In [ ]:
# Usuwam test aby nie zajmować pamięci
del(test)

Wczytanie poligonu geogograficznego miasta Nowy Jork wraz z dzielnicami.
Korzystam z wiedzy z serwisu https://www.kaggle.com/code/geowiz34/maps-of-nyc-airbnbs-with-python.
Dane geograficzne miasta Nowy Jork zostały pobrane z strony https://data.cityofnewyork.us/City-Government/Neighborhood-Tabulation-Areas/cpf4-rkhq i wyeksportowane do pliku nynta.csv

In [ ]:
# Pobieram współrzędne geograficzne granic pligonów miasta Nowy Jork wraz z dzielnicami
# W folderze data znajduje się plik nynta.zip, który należy rozpakować.
nbhoods = pd.read_csv('data/nynta.csv')
nbhoods

In [ ]:
# Sprawdzam czy kolumna NTACode w dataframe nbhood jest unikalna i czy można tej wartości z tej kolumny użyc jako klucz główny
cnt_no_uniq = (nbhoods["NTACode"].value_counts() > 1).sum()
cnt_no_uniq
# Wszystkie wartoci w kolumnie NTACode są unikalne

Przekonwertowanie danych geograficznych zawartych w DataFrame nbhoods (typu pandas) w kolumnie the_geom na typ danych odpowiedni dla geopandas, przez
utworzenie kolumny geom z współrzędnymi geograficznymi obszarów miasta Nowy Jork w formacie [Well-known text](https://en.wikipedia.org/wiki/Well-known_text_representation_of_geometry). Przekonwertowanie nbhoods z typu pandas DataFrame na geopandas GeoDataFrame. Typ danych kolumny geom będzię używany w dalszych analizach. Sposób konwertowania kolumny the_geom na well-know text został opisany w serwisie https://www.kaggle.com/code/geowiz34/maps-of-nyc-airbnbs-with-python.

In [ ]:
nbhoods.rename(columns={'NTAName':'neighbourhood'}, inplace=True)
nbhoods['geom'] = nbhoods['the_geom'].apply(wkt.loads)
nbhoods = geopandas.GeoDataFrame(nbhoods, geometry='geom')
nbhoods

In [ ]:
# Utworzenie DataFrame zawierajającego współrzędne geograficzne kolizji i identyfikatora kolizji
# geo_collisions będzię używany do nanoszenia punktów kolizji na mapę miasta Nowy Jork
geo_collisions = ny_collisions[["LONGITUDE","LATITUDE","COLLISION_ID","BOROUGH"]].loc[~ny_collisions["LOCATION"].isna()].reset_index(drop=True)
geo_collisions

In [ ]:
# Utworzenie kolumny point_collision zawierającego współrzędne geograficzne kolizji i przekonwertowanie z typu pandas DataFrame do geopandas GeoDataFrame
geo_collisions["point_collision"] = geopandas.GeoDataFrame(geopandas.points_from_xy(geo_collisions.LONGITUDE,geo_collisions.LATITUDE),columns=["point_collision"])
# Kolumna point_collision zawiera obiekty (punkty), które będą odrysowane na mapie miasta Nowy Jork
geo_collisions = geopandas.GeoDataFrame(geo_collisions, geometry='point_collision')
geo_collisions

Obszar miast Nowy Jork, znajduje się miej więcej pomiędzy długością geograficzną -73.67W a 74.3W, szerokością geograficzną 40.48N a 40.92N
W GeoDataFrame geo_collisions teorzę kolumnę "LocationNYC" z wartościami typu Boolen, która wszystkie współrzędne geograficzne z przedziału w/w obszaru oznacza jako True (znajdują się w Nowym Jorku lub w pobliżu) i False oznaczające współrzędne, które są błędne lub znajdują się poza obszarem wyznaczonym przez w/w obszar.
Kolumnę "LocationNYC", tworzę tylko na potrzeby wygodnego określenia, które punkty będą odrysowane na mapie

In [ ]:
geo_collisions["LocationNYC"] = False
geo_collisions["LocationNYC"] =  ((geo_collisions.LONGITUDE < -73.67) & (geo_collisions.LONGITUDE > -74.3) & (geo_collisions.LATITUDE > 40.48) & (geo_collisions.LATITUDE < 40.92))

In [ ]:
# Przykład punktów współrzędnych geograficznych, których nie narysuję na mapie miasta Nowy Jork
geo_collisions.loc[~geo_collisions["LocationNYC"]]

Przyporządkowanie punktów kolizji w geo_collisions do obszarów miasta Nowy Jork. Poniższa kod znajduje i przyporządkowuje lokalizacje do obszaru miasta Nowy Jork (NTACode w GeoDataFrame nbhoods). Przyporządkowanie trwa bardzo długo ok. 1h. W folderze data znajduje się plik geo.zip, który należy rozpakować. Plik zawiera gotowy GeoDataFrame geo_collisions

In [ ]:
if os.path.isfile("data/geo.json"):
    geo_collisions = geopandas.read_file("data/geo.json")
else:
    # bardzo długo ok 1h
    # Określenie obszarów miasta Nowy Jork na podstawie współrzędnych geograficznicznych
    # Kod dzielnicy Nowego Jorku z GeoDataFrame nbhoods["BoroCode"]
    geo_collisions["BoroCode"] = None
    # Kod obszaru wewnątrz dzielnic Nowego Jorku z GeoDataFrame nbhoods["NTACode"]
    geo_collisions["NTACode"] = None
    for i in tqdm(geo_collisions.loc[geo_collisions["LocationNYC"]].index):
        data = nbhoods[["BoroCode","NTACode"]].loc[nbhoods["geom"].contains(geo_collisions.loc[i,"point_collision"])]
        l = len(data)
        if l == 1:
            # Jednoznaczne przyporządkowanie do jednego obszaru miasta
            geo_collisions.at[i,"BoroCode"] = int(data["BoroCode"].values[0])
            geo_collisions.at[i,"NTACode"] = data["NTACode"].values[0]
        if l > 1:
            # Testowo, lokalizacja określająca kilka obszarów miasta, być może oznacza błąd w danych nbhoods
            test.at[i,"BoroCode"] = -l
    geo_collisions.to_file("data/geo.json",driver="GeoJSON",index=False)

In [ ]:
# Sprawdzenie lokalizacji które są w obszarze miasta Nowy Jork, a zostały zaznaczone jak nie należące do
# żadnego obszaru miasta
geo_collisions.loc[geo_collisions["BoroCode"].isna() & geo_collisions["LocationNYC"]]

Jak widać na mapie poniżej (czerwone kropki) są to lokalizacje w obszarach wodnych (również na mostach) lub w ich pobliżu i obszary poza miastem Nowy Jork.
Problem wynika z faktu że GeoDataFrame nbnbhoods nie wprowadza obszarów wodnych do miasta Nowy Jork, dodatkowo dochodzi problem z dokładnością lokalizacj przy brzegach obszarów wodnych.

In [ ]:
fig,ax = plt.subplots(1,1, figsize=(15,15))
base = nbhoods.plot(color="orange",alpha=0.5,edgecolor='black',ax=ax)
geo_collisions.loc[geo_collisions["BoroCode"].isna() & geo_collisions["LocationNYC"]].plot(ax=base, color="red", markersize=8)

W GeoDataFrame geo_collisions mamy dwie kolumny:
* BOROUGH: z nazwą dzielnicy, która pochodzi z pandas DataFrame ny_collisions["BOROUGH"]
* BoroCode: z kodem dzielnicy w postaci liczby całkowitej, która pochodzi z przyporządkowania lokalizacji ny_collision[["LONGITUDE","LATITUDE"]] do obszaru w GeoDataFrame nbhoods.

Lokalizacje które należą do obszaru miasta Nowy Jork, ponieważ mają określoną nazwę dzielnicy w kolumnie BOROUGH, a nie mają określonej wartości kodu dzielnicy w kolumnie BoroCode (obszary wodne i pobliże wody), można uzupełnić przez przypisanie kodu dzielnicy do BoroCode na podstawie wartosci w kolumnie BOROUGH. Niestety wartość w kolumnie NTACode pozostanie nie uzupełniona. Przypisanie BOROUGH do BoroCode, oparte jest na zaufaniu że lokalizacja dzielnicy Nowego Jorku jest prawidłowa.
W ten sposób dołączymy część lokalizacji do późniejszych analiz, opartych na całych dzielnicach miasta Nowy Jork. Mniejszych obszarów nie będzie można dołączyć, ponieważ nie mają uzupełnionej kolumny NTACode.

In [ ]:
# Jakie unikalne wartości mamy w kolumnie BOROUGH geodataframe geo_collisions
geo_collisions["BOROUGH"].unique()

In [ ]:
# kody dzielnic miasta Nowy Jork z geodataframe nbhoods
nbhoods[["BoroName","BoroCode"]].groupby("BoroName").BoroCode.unique()

## Analiza danych

In [ ]:
# Rysuję obszary miasta Nowy Jork na podstwaie danych geograficznych zawartych w DataFrame nbhoods
fig,ax = plt.subplots(1,1, figsize=(15,15))
nbhoods.plot(color="orange",alpha=0.5,edgecolor='black',ax=ax)